## 作成したデータから重回帰分析を実施し、それぞれの特徴量の重要度を評価する

In [ ]:
!pip install -q shap

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
import statsmodels.api as sm

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

import shap

In [ ]:
# ドライブをマウント
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# データの読み込み
import pandas as pd
df = pd.read_csv("learning_Li_battery_data.csv")

In [ ]:
df.head()

,num_steps,nelements,max_delta_volume,average_voltage,capacity_grav,fracA_discharge,stability_charge,minimum CN_VoronoiNN,maximum CN_VoronoiNN,range CN_VoronoiNN,...,minimum local difference in GSvolume_pa,maximum local difference in GSvolume_pa,mean local difference in GSvolume_pa,minimum local difference in SpaceGroupNumber,maximum local difference in SpaceGroupNumber,avg_dev local difference in SpaceGroupNumber,0-norm,2-norm,frac s valence electrons,frac p valence electrons
0,2,3,0.008674,3.645088,137.171754,0.250000,0.037008,5.999755,10.195544,4.195789,...,1.140000,7.084399,2.644168,113.035858,212.940334,33.974522,4,0.630840,0.292135,0.359551
1,1,3,0.003674,3.555958,69.825436,0.250000,0.089504,5.999351,8.565770,2.566419,...,1.215000,6.936504,3.498748,156.870787,216.998680,22.156506,4,0.607362,0.298851,0.367816
2,1,4,0.078912,3.637514,148.887306,0.142857,0.067563,4.363747,11.904116,7.540370,...,1.966086,22.455000,6.001459,10.000000,213.100189,44.926378,4,0.699702,0.274390,0.481707
3,1,3,0.024693,2.901634,184.256618,0.181818,0.078001,5.979264,9.364866,3.385601,...,1.208721,7.730853,3.161494,77.542331,188.099741,37.717392,4,0.583095,0.350000,0.400000
4,1,5,0.071289,3.757691,113.645161,0.100000,0.045275,4.070083,10.731594,6.661512,...,1.214808,13.465238,4.284699,10.000000,212.979362,48.159064,5,0.695000,0.314159,0.504425


## 学習用データの準備

In [ ]:
# 数値データの抽出
df_num = df.select_dtypes(include=["float64", "int64"])

In [ ]:
# 目的変数の設定
target = "capacity_grav"

In [ ]:
# 入力変数と出力変数の切り分け
y = df_num[target]
x = df_num.drop(labels=[target], axis=1)

In [ ]:
x

,num_steps,nelements,max_delta_volume,average_voltage,fracA_discharge,stability_charge,minimum CN_VoronoiNN,maximum CN_VoronoiNN,range CN_VoronoiNN,mean CN_VoronoiNN,...,minimum local difference in GSvolume_pa,maximum local difference in GSvolume_pa,mean local difference in GSvolume_pa,minimum local difference in SpaceGroupNumber,maximum local difference in SpaceGroupNumber,avg_dev local difference in SpaceGroupNumber,0-norm,2-norm,frac s valence electrons,frac p valence electrons
0,2,3,0.008674,3.645088,0.250000,0.037008,5.999755,10.195544,4.195789,8.009345,...,1.140000,7.084399,2.644168,113.035858,212.940334,33.974522,4,0.630840,0.292135,0.359551
1,1,3,0.003674,3.555958,0.250000,0.089504,5.999351,8.565770,2.566419,7.411034,...,1.215000,6.936504,3.498748,156.870787,216.998680,22.156506,4,0.607362,0.298851,0.367816
2,1,4,0.078912,3.637514,0.142857,0.067563,4.363747,11.904116,7.540370,8.852230,...,1.966086,22.455000,6.001459,10.000000,213.100189,44.926378,4,0.699702,0.274390,0.481707
3,1,3,0.024693,2.901634,0.181818,0.078001,5.979264,9.364866,3.385601,7.566882,...,1.208721,7.730853,3.161494,77.542331,188.099741,37.717392,4,0.583095,0.350000,0.400000
4,1,5,0.071289,3.757691,0.100000,0.045275,4.070083,10.731594,6.661512,8.518036,...,1.214808,13.465238,4.284699,10.000000,212.979362,48.159064,5,0.695000,0.314159,0.504425
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2386,1,2,0.022063,2.783307,0.040000,0.137107,5.984715,9.405782,3.421067,8.195193,...,1.271434,3.905000,2.155490,70.653291,217.000000,64.813276,2,0.745356,0.352941,0.470588
2387,1,2,0.091835,3.273561,0.166667,0.082159,4.937797,11.932746,6.994949,8.954779,...,0.499739,3.905000,1.985509,27.770406,217.000000,63.972552,2,0.761577,0.350877,0.491228
2388,1,2,0.052291,3.497992,0.125000,0.012942,5.289024,13.159879,7.870855,9.823587,...,0.701150,3.905000,1.941508,38.962735,217.000000,62.349029,2,0.769309,0.350000,0.500000
2389,2,2,0.015606,3.531304,0.086957,0.029818,5.461302,12.092674,6.631372,9.631077,...,0.537198,3.905000,1.944278,29.851992,217.000000,62.261096,2,0.769309,0.350000,0.500000


In [ ]:
# データ数確認
type(x), x.shape, type(y), y.shape

(pandas.core.frame.DataFrame, (2391, 72), pandas.core.series.Series, (2391,))

In [ ]:
# 標準化
x_scaled = scaler.fit_transform(x)
x_scaled = pd.DataFrame(x_scaled, columns=x.columns)
x_scaled

,num_steps,nelements,max_delta_volume,average_voltage,fracA_discharge,stability_charge,minimum CN_VoronoiNN,maximum CN_VoronoiNN,range CN_VoronoiNN,mean CN_VoronoiNN,...,minimum local difference in GSvolume_pa,maximum local difference in GSvolume_pa,mean local difference in GSvolume_pa,minimum local difference in SpaceGroupNumber,maximum local difference in SpaceGroupNumber,avg_dev local difference in SpaceGroupNumber,0-norm,2-norm,frac s valence electrons,frac p valence electrons
0,1.403948,-0.100498,-0.794605,0.042142,0.968234,-0.207690,1.056008,-0.790760,-1.232197,-0.689800,...,-0.326309,-0.561811,-0.602367,1.348110,0.193940,-0.771538,0.786165,-1.072182,-0.739486,-1.553734
1,-0.434409,-0.100498,-0.897402,-0.011036,0.968234,-0.082707,1.055559,-1.917096,-2.190653,-1.221655,...,-0.255285,-0.586103,-0.228408,2.309615,0.409644,-1.700847,0.786165,-1.502013,-0.562161,-1.457224
2,-0.434409,1.459831,0.649298,0.037622,-0.247453,-0.134944,-0.761561,0.390033,0.735210,0.059466,...,0.455982,1.962861,0.866763,-0.911949,0.202436,0.089661,0.786165,0.188547,-1.208020,-0.127405
3,-0.434409,-0.100498,-0.465307,-0.401429,0.194615,-0.110093,1.033243,-1.364841,-1.708780,-1.083118,...,-0.261231,-0.455629,-0.375989,0.569571,-1.126360,-0.477218,0.786165,-1.946294,0.788406,-1.081437
4,-0.434409,3.020159,0.492599,0.109324,-0.733728,-0.188008,-1.087815,-0.420296,0.218233,-0.237609,...,-0.255467,0.486263,0.115519,-0.911949,0.196014,0.343863,2.129781,0.102453,-0.157944,0.137849
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2386,-0.434409,-1.660827,-0.519363,-0.472027,-1.414513,0.030628,1.039299,-1.336564,-1.687918,-0.524594,...,-0.201843,-1.084038,-0.816210,0.418462,0.409714,1.653465,-1.901068,1.024373,0.866066,-0.257234
2387,-0.434409,-1.660827,0.914953,-0.179524,0.022699,-0.100195,-0.123804,0.409818,0.414373,0.150624,...,-0.932626,-1.084038,-0.890592,-0.522160,0.409714,1.587355,-1.901068,1.321353,0.811568,-0.016238
2388,-0.434409,-1.660827,0.102032,-0.045621,-0.450068,-0.264986,0.266402,1.257889,0.929614,0.922933,...,-0.741893,-1.084038,-0.909847,-0.276660,0.409714,1.459689,-1.901068,1.462909,0.788406,0.086185
2389,1.403948,-1.660827,-0.652104,-0.025746,-0.881725,-0.224808,0.457798,0.520344,0.200504,0.751804,...,-0.897153,-1.084038,-0.908635,-0.476502,0.409714,1.452775,-1.901068,1.462909,0.788406,0.086185


## 重回帰分析

In [ ]:
# 分析
multi_ols = sm.OLS(y, sm.add_constant(x_scaled))

result = multi_ols.fit()

In [ ]:
# 結果の確認
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          capacity_grav   R-squared:                       0.930
Model:                            OLS   Adj. R-squared:                  0.928
Method:                 Least Squares   F-statistic:                     456.7
Date:                Tue, 07 May 2024   Prob (F-statistic):               0.00
Time:                        13:50:57   Log-Likelihood:                -10468.
No. Observations:                2391   AIC:                         2.107e+04
Df Residuals:                    2322   BIC:                         2.147e+04
Df Model:                          68                                         
Covariance Type:            nonrobust                                         
===================================================================================================================
                                                      coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------------
const                                             136.8954      0.400    342.078      0.000     136.111     137.680
num_steps                                           5.8531      0.486     12.055      0.000       4.901       6.805
nelements                                          51.9532      4.751     10.936      0.000      42.637      61.269
max_delta_volume                                    3.4930      0.509      6.859      0.000       2.494       4.492
average_voltage                                    -0.0639      0.487     -0.131      0.896      -1.019       0.892
fracA_discharge                                   102.1548      1.015    100.678      0.000     100.165     104.145
stability_charge                                   -1.3244      0.554     -2.391      0.017      -2.411      -0.238
minimum CN_VoronoiNN                               -1.0279      1.669     -0.616      0.538      -4.301       2.245
maximum CN_VoronoiNN                                1.0299      0.896      1.150      0.250      -0.726       2.786
range CN_VoronoiNN                                  1.4208      0.752      1.890      0.059      -0.053       2.895
mean CN_VoronoiNN                                   5.2648      2.352      2.238      0.025       0.652       9.878
avg_dev CN_VoronoiNN                               -9.3818      2.301     -4.077      0.000     -13.895      -4.869
mean absolute deviation in relative bond length    -1.5910      2.460     -0.647      0.518      -6.415       3.233
max relative bond length                            0.8453      0.978      0.865      0.387      -1.072       2.762
min relative bond length                            2.8115      2.212      1.271      0.204      -1.527       7.150
minimum neighbor distance variation                -0.9397      0.717     -1.310      0.190      -2.346       0.467
maximum neighbor distance variation                 0.9192      0.663      1.387      0.166      -0.380       2.219
range neighbor distance variation                   1.4413      0.563      2.559      0.011       0.337       2.546
avg_dev neighbor distance variation                 4.5950      1.024      4.489      0.000       2.588       6.602
mean ordering parameter shell 1                     1.0706      1.178      0.909      0.363      -1.239       3.380
mean ordering parameter shell 2                    -0.3235      1.645     -0.197      0.844      -3.550       2.903
mean ordering parameter shell 3                     0.5750      1.146      0.502      0.616      -1.672       2.822
max packing efficiency                            -15.5954      2.403     -6.489      0.000     -20.308     -10.883
minimum local difference in Number                 -5.3135      1.9

In [ ]:
# 自由度調整済み決定係数
result.rsquared_adj

0.9283983350886127

In [ ]:
# 偏回帰係数
result.params

const                                           136.895380
num_steps                                         5.853082
nelements                                        51.953158
max_delta_volume                                  3.492955
average_voltage                                  -0.063864
                                                   ...    
avg_dev local difference in SpaceGroupNumber     -3.199546
0-norm                                          -62.848802
2-norm                                           -5.582475
frac s valence electrons                        -12.614383
frac p valence electrons                         22.422679
Length: 73, dtype: float64

In [ ]:
# p 値
result.pvalues

const                                           0.000000e+00
num_steps                                       1.666300e-32
nelements                                       3.525853e-27
max_delta_volume                                8.862130e-12
average_voltage                                 8.957417e-01
                                                    ...     
avg_dev local difference in SpaceGroupNumber    4.365997e-02
0-norm                                          3.554949e-29
2-norm                                          2.548218e-06
frac s valence electrons                        3.191450e-17
frac p valence electrons                        1.048111e-16
Length: 73, dtype: float64

In [ ]:
# p値が5%以上の要素をフィルタリング
result.pvalues[result.pvalues >= 0.05]

average_voltage                                    0.895742
minimum CN_VoronoiNN                               0.538042
maximum CN_VoronoiNN                               0.250312
range CN_VoronoiNN                                 0.058894
mean absolute deviation in relative bond length    0.517863
max relative bond length                           0.387316
min relative bond length                           0.203900
minimum neighbor distance variation                0.190261
maximum neighbor distance variation                0.165603
mean ordering parameter shell 1                    0.363443
mean ordering parameter shell 2                    0.844145
mean ordering parameter shell 3                    0.615947
mean local difference in Number                    0.905729
minimum local difference in MeltingT               0.057296
maximum local difference in MeltingT               0.443135
mean local difference in MeltingT                  0.075874
avg_dev local difference in MeltingT    

In [ ]:
# t 値
result.tvalues

const                                           342.077917
num_steps                                        12.055172
nelements                                        10.935966
max_delta_volume                                  6.858967
average_voltage                                  -0.131057
                                                   ...    
avg_dev local difference in SpaceGroupNumber     -2.018442
0-norm                                          -11.366872
2-norm                                           -4.715974
frac s valence electrons                         -8.505394
frac p valence electrons                          8.362042
Length: 73, dtype: float64